Описание
Данный блокнот проводит аналогичный анализ, но без учета стран. Фокус на распределении cn и их связи с сервисом bablosoft.

Этапы
Подключение к ClickHouse:

Аналогично первому блокноту, используется безопасное соединение для получения данных.
Формирование первого запроса:

Получение данных о cn и их связи с bablosoft с аналогичными фильтрами:
Период: август 2024 года.
Исключение запросов из России.
Исключение ошибочных запросов.
Анализ данных:

Подсчет общего количества уникальных cn.
Подсчет количества cn, связанных с сервисом.

In [1]:
#Скрипт используется для поиска сразну нескольких сервисов: bablosoft, ipinfo, zennolab
#Для того, чтобы найти процент cn с сервисом по странам, достаточно подствить эти значеня в метод LIKE в запросе SQL

In [2]:
import os
from clickhouse_driver import Client
import pandas as pd

# Получаем пароль из переменной окружения
password = os.getenv("CLICKHOUSE_PASSWORD")

# Проверяем, что пароль не является None
if password is None:
    raise ValueError("Переменная окружения CLICKHOUSE_PASSWORD не установлена")

client = Client(
    host='ch.infra.zprx.net',
    port=2044,
    user='zprx_analytics',
    password=password,
    secure=True,
    verify=False,
    database='zprx'
)

try:
    query = """
SELECT cn,
       CASE 
           WHEN req_host LIKE '%bablosoft%' THEN cn 
           ELSE NULL 
       END AS cn_with_service
FROM 
     zprx.zproxy_reqs_v1__merge
WHERE toYYYYMM(ts_millis) = toYYYYMM(toDate('2024-08-01'))
  AND geoip2_country(client_ip) != 'Russia'
  AND error = '0'
GROUP BY cn, 
         cn_with_service
    """

    # Execute the query and store the result in DataFrame
    result = client.execute(query)
    cn_per_service = pd.DataFrame(result, columns=["cn", "cn_with_service"])

    # Print the results in table format
    print(cn_per_service)

finally:
    client.disconnect()

              cn cn_with_service
0     AFeQOePkdz            None
1     Okr8yuv7hC            None
2     D9lWPbTuLC            None
3     FdBeEsQAi8            None
4     hEQk19O9zO            None
...          ...             ...
9970  RSlXR6iVnK            None
9971  XESAE1rfIi      XESAE1rfIi
9972  hCGAgeTH4R            None
9973  Ru7tBipiMm            None
9974  8EfgDH7VJN            None

[9975 rows x 2 columns]


In [3]:
import os
from clickhouse_driver import Client
import pandas as pd

# Получаем пароль из переменной окружения
password = os.getenv("CLICKHOUSE_PASSWORD")

# Проверяем, что пароль не является None
if password is None:
    raise ValueError("Переменная окружения CLICKHOUSE_PASSWORD не установлена")

client = Client(
    host='ch.infra.zprx.net',
    port=2044,
    user='zprx_analytics',
    password=password,
    secure=True,
    verify=False,
    database='zprx'
)

try:
    query = """
SELECT user_id,
       cn
FROM zprx_ds.device
WHERE user_id is not NUll
GROUP BY user_id,
         cn
    """

    # Execute the query and store the result in DataFrame
    result = client.execute(query)
    cn_per_user_id = pd.DataFrame(result, columns=["user_id", "cn"])

    # Print the results in table format
    print(cn_per_user_id)

finally:
    client.disconnect()

          user_id          cn
0      rbwnJagmNN  Cb9hjRB849
1      EI38Acy1Fu  mW3r5vCPDq
2      SDRGfJMG0v  MbYBbZMMZ4
3      pOu6lPJB7O  JyW7Ax7yo2
4      kEVUaSXFTd  qKI4DRKVch
...           ...         ...
70290  NOP9RAwaNo  vlJbrXnnce
70291  MjOvWPJctn  VFEm6CHvos
70292  WckroSFUsK  IAGNs9LuqA
70293  Yzz1odAD28  p8trySDwOz
70294  XoJdMcvjbY  jEjgMeckvY

[70295 rows x 2 columns]


In [4]:
import os
from clickhouse_driver import Client
import pandas as pd

# Получаем пароль из переменной окружения
password = os.getenv("CLICKHOUSE_PASSWORD")

# Проверяем, что пароль не является None
if password is None:
    raise ValueError("Переменная окружения CLICKHOUSE_PASSWORD не установлена")

client = Client(
    host='ch.infra.zprx.net',
    port=2044,
    user='zprx_analytics',
    password=password,
    secure=True,
    verify=False,
    database='zprx'
)

try:
    query = """
SELECT anyHeavy(geoip2_country(ipv4)) country,
       cn
FROM zprx.iphistory_v1__merge
WHERE toYYYYMM(ts) = toYYYYMM(toDate('2024-08-01'))
GROUP BY cn
HAVING country != 'Russia'
    """

    # Execute the query and store the result in DataFrame
    result = client.execute(query)
    ip_per_country = pd.DataFrame(result, columns=["country", "cn"])

    # Print the results in table format
    print(ip_per_country)

finally:
    client.disconnect()

            country          cn
0     United States  HoB1xEcq4M
1       Philippines  NxbLAPfH4G
2            Brazil  bq7REjTWjl
3             India  8Ll7ToEW2U
4     United States  SDRxA8EARf
...             ...         ...
8330  United States  xYcFdiZZOd
8331         Turkey  HSbceY5v2Z
8332        Ukraine  8Nsd1SSohc
8333         Brazil  t3qevdF88d
8334         Brazil  bwcNNIGxHr

[8335 rows x 2 columns]


In [5]:
df_temp = pd.merge(cn_per_service, cn_per_user_id, on='cn', how='left')

# Затем объединяем результат с df_3
cn_user_country = pd.merge(df_temp, ip_per_country, on='cn', how='left')

In [6]:
cn_user_country

,cn,cn_with_service,user_id,country
0,AFeQOePkdz,None,YGnEYF7BLE,United States
1,Okr8yuv7hC,None,hL6j99BoZI,Kazakhstan
2,D9lWPbTuLC,None,WQF1pErJ6H,Germany
3,FdBeEsQAi8,None,J6XhhwsTkq,Brazil
4,hEQk19O9zO,None,pNhTDDgBcx,Brazil
...,...,...,...,...
9970,RSlXR6iVnK,None,bNVhdYAMYL,Thailand
9971,XESAE1rfIi,XESAE1rfIi,pTv7EG6PBj,United States
9972,hCGAgeTH4R,None,N5MF5oTx8G,United Kingdom
9973,Ru7tBipiMm,None,EI38Acy1Fu,Thailand


In [7]:
cn_user_country_grouped = cn_user_country.groupby(['country']).agg(
    total_cn=('cn', 'count'),
    total_cn_with_service=('cn_with_service', lambda x: x.notna().sum())
).reset_index()

# Вычисляем процент cn_with_bablosoft от общего числа cn и округляем до десятых
cn_user_country_grouped['cn_with_service_percent'] = (cn_user_country_grouped['total_cn_with_service'] / cn_user_country_grouped['total_cn'] * 100).round(0)

In [8]:
cn_user_country_grouped['cn_with_service_percent'] = cn_user_country_grouped['cn_with_service_percent'].astype('int')

In [9]:
# Группируем данные по странам и считаем количество уникальных user_id с cn и cn_with_bablosoft
df_grouped_user_id = cn_user_country.groupby('country').agg(
    total_user_ids=('user_id', 'nunique'),
    user_ids_with_service=('user_id', lambda x: x[cn_user_country.loc[x.index, 'cn_with_service'].notna()].nunique())
).reset_index()

# Вычисляем процент и округляем до десятых
df_grouped_user_id['service_percent'] = (df_grouped_user_id['user_ids_with_service'] / df_grouped_user_id['total_user_ids'] * 100).round(1)
df_grouped_user_id['service_percent'] = df_grouped_user_id['service_percent'].astype(int)


In [10]:
cn_user_country_grouped.sort_values(by='cn_with_service_percent', ascending=False)

,country,total_cn,total_cn_with_service,cn_with_service_percent
75,Singapore,39,17,44
18,Colombia,441,187,42
54,Moldova,62,24,39
77,South Korea,78,30,38
68,Peru,59,20,34
...,...,...,...,...
58,Namibia,1,0,0
59,Netherlands,24,0,0
60,New Zealand,1,0,0
61,Nicaragua,2,0,0


In [11]:
df_grouped_user_id.sort_values(by='service_percent', ascending=False)

,country,total_user_ids,user_ids_with_service,service_percent
77,South Korea,11,6,54
76,Slovenia,2,1,50
84,Tunisia,2,1,50
72,Puerto Rico,7,3,42
85,Turkey,71,20,28
...,...,...,...,...
58,Namibia,1,0,0
59,Netherlands,12,0,0
60,New Zealand,1,0,0
61,Nicaragua,1,0,0


In [12]:
cn_user_country_grouped = cn_user_country_grouped.sort_values(by='cn_with_service_percent', ascending=False)

In [13]:
df_grouped_user_id = df_grouped_user_id.sort_values(by='service_percent', ascending=False)

In [14]:
cn_user_country_grouped.to_csv('доля_cn_с_bablosoft_по_странам(ipv4).csv', index=False)

In [15]:
df_grouped_user_id.to_csv('доля_user_id_с_bablosoft_по_странам(ipv4).csv', index=False)

Разделение по CN:

Высокий процент CN с услугой наблюдается в странах, таких как Сингапур (44%), Колумбия (42%), и Молдова (39%).
В ряде стран, включая Нидерланды, Намибию, и Афганистан, услуги не использовались (0%).
Разделение по User ID:

Лидерами по проценту пользователей с услугой стали Южная Корея (54%), Словения и Тунис (по 50%).
В странах, таких как Нидерланды, Никарагуа, и Афганистан, услугой не воспользовались (0%).